In [9]:
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr, field_validator
from datetime import date
import json

app = FastAPI()

class Subscriber(BaseModel):
    last_name: str
    first_name: str
    birth_date: date
    phone: str
    email: EmailStr

    @field_validator("last_name", "first_name")
    @classmethod
    def check_name(cls, value: str) -> str:
        if not value:
            raise ValueError("Пустая строка")
        if not value[0].isupper():
            raise ValueError("Первая буква должна быть заглавной")
        for ch in value:
            if not ("А" <= ch <= "я" or ch in "Ёё"):
                raise ValueError("Допускаются только кириллические буквы")
        return value

    @field_validator("phone")
    @classmethod
    def check_phone(cls, value: str) -> str:
        if not (value.isdigit() and len(value) == 11):
            raise ValueError("Телефон должен содержать 11 цифр")
        return value

    @field_validator("birth_date")
    @classmethod
    def check_birth_date(cls, value: date) -> date:
        if value > date.today():
            raise ValueError("Дата рождения не может быть в будущем")
        return value

    @field_validator("email")
    @classmethod
    def check_email(cls, value: EmailStr) -> EmailStr:
        if len(value) > 320:
            raise ValueError("Слишком длинный email")
        return value


@app.post("/subscribe")
def create_subscriber(subscriber: Subscriber):
    data = subscriber.model_dump(mode="json")
    with open("subscriber.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    return {"status": "ok", "data": data}
